## Reasoning and Acting (ReAct)​

Agent to implement ReAct logic

In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import AzureChatOpenAI

In [3]:
%pip install pypdf
%pip install pycryptodome
%pip install pymupdf

  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/3c/60/eccdd92dd4af3e4bea6d6a342f7588c618a15b9bec4b968af581e498bcc4/pypdf-4.3.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 5.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for pycryptodome from https://files.pythonhosted.org/packages/24/80/56a04e2ae622d7f38c1c01aef46a26c6b73a2ad15c9705a8e008b5befb03/pycryptodome-3.20.0-cp35-abi3-macosx_10_9_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.4 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip is available: 23.2.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for pymupdf fr

In [4]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("data/Underwriting Guidelines 319E 1.pdf")

data = loader.load()

data[0]

Document(page_content='BMO Insurance\nEffective Date:\nFebruary 26, 2024\n319E (2024/02/26)\nUnderwriting \nGuidelines\n', metadata={'source': 'data/Underwriting Guidelines 319E 1.pdf', 'file_path': 'data/Underwriting Guidelines 319E 1.pdf', 'page': 0, 'total_pages': 27, 'format': 'PDF 1.7', 'title': 'Underwriting Guidelines', 'author': 'BMO Insurance', 'subject': '', 'keywords': '', 'creator': 'Adobe InDesign 19.1 (Macintosh)', 'producer': 'Adobe PDF Library 17.0', 'creationDate': "D:20240126101951-05'00'", 'modDate': "D:20240126111742-05'00'", 'trapped': '', 'encryption': 'Standard V5 R6 256-bit AES'})

In [6]:
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.tools.retriever import create_retriever_tool

embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002", openai_api_version="2023-05-15")
vector = FAISS.from_documents(data, embeddings)
retriever = vector.as_retriever()

retriever_tool = create_retriever_tool(
    retriever,
    "bmo_search",
    "Search for information about insurance underwriting. For any questions about insurance, you must use this tool!",
)

In [7]:
tools = [retriever_tool, TavilySearchResults(max_results=1)]

In [8]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

In [16]:
# Choose the LLM to use
llm = AzureChatOpenAI(
    openai_api_version="2024-06-06",
    azure_deployment="gpt-4o",
)

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [17]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [11]:
# agent_executor.invoke({"input": "what is LangChain?"})

In [18]:
agent_executor.invoke(
    {"input": "What are the requirements for an Male applicant aged  70 for $9M ?"}
)



> Entering new AgentExecutor chain...


NotFoundError: Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}